## Diminutive Suffix Productivity
Juan Berrios | jeb358@pitt.edu | Last updated: February 24, 2020

**Summary:**

- The purpose of the code included in this notebook is to build `DataFrame` objects from the `.txt` files in the corpus directories. Section 1 includes the necessary preparations. 

**1. Preparation**

- Loading libraries and additional settings:

In [1]:
#Importing libraries
import nltk, glob, re, pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

#Turning pretty print off:
%pprint

#Releasing all output:                                            
from IPython.core.interactiveshell import InteractiveShell #Prints all commands rather than the last one.
InteractiveShell.ast_node_interactivity = "all"

Pretty printing has been turned OFF


- The functions below are meant to streamline the processing of the corpus. They are all defined based on the code in Section 2. 

In [2]:
def toDF(fname):
    """Turns tab-delimited file into a data frame"""
    cols = ['SourceID', 'TokenID', 'Word', 'Lemma', 'POS']
    df = pd.read_csv(fname,sep='\t',encoding ='iso-8859-1',skiprows=[0],header=None,names=cols)
    return df

In [3]:
def add_variety(df,variety):
    """Adds a column specifying a given variety of Spanish to a data frame"""
    df['Variety'] = variety

**2. Import files**

- The text files are very large. For testing purpose, I'll use only one of the text files as a start. The `.txt` files are tab delimited which makes my job a little easier. The columns correspond to an ID for the source text, an ID for the token, the token (word), the lemma, and the POS. I will hence use those for column names. 

In [4]:
fname = 'C:/Users/Juan/Documents/LING2340/Diminutive-Suffix-Productivity/private/data/wlp_ES-sbo/es-b-0.txt'

In [5]:
cols = ['SourceID', 'TokenID', 'Word', 'Lemma', 'POS']

In [6]:
#First row ignored because it corresponds to an identifier for the .txt file.

df = pd.read_csv(fname,sep='\t',encoding ='iso-8859-1',skiprows=[0],header=None,names=cols) 

In [7]:
df.shape #It's a very large file. It will get much smaller once I start extracting the tokens of interest.

(27345213, 5)

In [8]:
df.head()   #The lemma column will be useful when I need to aggregations that put lowecase and uppercase 
            #as well as plural and singular forms together.

,SourceID,TokenID,Word,Lemma,POS
0,431270,2206403096,Este,este,dd-
1,431270,2206403097,es,ser,vip-3s
2,431270,2206403098,un,un,li-ms
3,431270,2206403099,blog,blog,nms
4,431270,2206403100,de,de,e


In [9]:
df.tail()

,SourceID,TokenID,Word,Lemma,POS
27345208,676060,2511862343,mas,mas,cc
27345209,676060,2511862344,informacion,información,n
27345210,676060,2511862345,visite,visitar,vsp-1/3s
27345211,676060,2511862346,www.DineroAbundancia.com,www.dineroabundancia.com,n
27345212,676060,2511862347,.,$.,y


In [10]:
df.sample(5)

,SourceID,TokenID,Word,Lemma,POS
3497259,469430,268135739,2000,2000,m$
18986538,598630,2413054034,ya,ya,r
17481929,590320,2157047554,la,la,ld-fs
5578699,490480,2220894016,",","$,",y
26751384,674240,1395878593,Ocurrió,ocurrir,vis-3s


In [11]:
df['POS'].unique() #These POS tags are not very transparent, but it's a good start.

array(['dd-', 'vip-3s', 'li-ms', 'nms    ', 'e', 'nfp    ', 'y',
       'nfs    ', 'nmp    ', 'vsp-1/3s', 'n', 'cc', 'o', 'r', 'vps-ms',
       'ld-fs', 'm$', 'vc-1/3s', 'vr', 'po', 'li-fs', 'ld-mp', 'vip-3p',
       'ld-ms', 'jms    ', 'vpp', 'mc', 'vps-fs', 'j', 'vii-1/3s', 'e_21',
       'x', nan, 'cs', 'ld', 'v', 'cc-', 'ld-fp', 'pi-0cn', 'dd',
       'dxmp-ind-', 'vip-2s', 'dp-', 'vsp-3p', 'pi-0ms', 'jfp    ',
       'cS_21', 'cS_22', 'jmp    ', 'vsp-2s', 'vip-1s', 'vsp-1p', 'ps',
       'vip-1p/vis-1p', 'vip-1p', 'pd-3cs', 'vif-1p', 'vps-mp', 'vif-3s',
       'vif-3p', 'jfs    ', 'dxfs-ind-', 'vsj-1/3s', 'i', 'pi-3ms',
       'vis-3s', 'b', 'p', 'np', 'pr-3cn"', 'px', 'dxms-ind-', 'pi-3cs',
       'dxfs-', 'pr-3cs', 'px-ms', 'dxfp-ind-', 'vis-3p', 'li-mp', 'pi',
       'vsi-3p', 'px-mp', 'vsi-1/3s', 'pq-3cn"', 'vif-2s', 'vif-2p',
       'vsp-2p', 'vip-2p', 'vpp-00', 'vm-2p', 'vis-1p', 'dxcs-ind-',
       'pr-3cp', 'dxcs-dem-', 'vif-1s', 'vc-1p', 'cC_21', 'cC_22',
       'vps-fp',

In [12]:
df['Variety'] = 'ES' #Time to add a column for the variety of Spanish. In this case Spain.

In [13]:
df.head()

,SourceID,TokenID,Word,Lemma,POS,Variety
0,431270,2206403096,Este,este,dd-,ES
1,431270,2206403097,es,ser,vip-3s,ES
2,431270,2206403098,un,un,li-ms,ES
3,431270,2206403099,blog,blog,nms,ES
4,431270,2206403100,de,de,e,ES


In [14]:
df.tail()

,SourceID,TokenID,Word,Lemma,POS,Variety
27345208,676060,2511862343,mas,mas,cc,ES
27345209,676060,2511862344,informacion,información,n,ES
27345210,676060,2511862345,visite,visitar,vsp-1/3s,ES
27345211,676060,2511862346,www.DineroAbundancia.com,www.dineroabundancia.com,n,ES
27345212,676060,2511862347,.,$.,y,ES


In [15]:
df.sample(5)

,SourceID,TokenID,Word,Lemma,POS,Variety
11869609,547760,1151256381,calidad,calidad,nfs,ES
1842942,452310,2135062343,gestado,gestar,vps-ms,ES
10277289,535460,341803833,película,película,nfs,ES
25856292,664920,1607211213,crecimiento,crecimiento,nms,ES
2687822,463000,538042731,imitación,imitación,nfs,ES
